In [ ]:
!pip install rdkit-pypi
!pip install XlsxWriter
!pip install gdown
!pip install matminer
!pip install skorch
!pip install flask


In [55]:
import os
import pandas as pd
import requests
from datetime import datetime
from sklearn.model_selection import train_test_split
from matminer.datasets import load_dataset
from matminer.featurizers.conversions import StructureToComposition
from matminer.featurizers.composition import ElementProperty
from skorch import NeuralNetRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline

# URL with the shareable link to your Google Sheets document
google_sheets_url = "https://docs.google.com/spreadsheets/d/1jdoToY-lbibVxI0mcGwPf6LkkI786JJQwDWVzQQ7Glo/edit#gid=159703025"

# Read data
df = pd.read_csv(f"{google_sheets_url.replace('/edit#gid=', '/gviz/tq?tqx=out:csv&gid=')}")

# Convert the 'Date' column to datetime format
df['Flow Tag Date'] = pd.to_datetime(df['Flow Tag Date'], format='%d-%b-%y %H:%M:%S')

# Replace values above 125 kW with 0 in the 'EL_Solar_BusBarn_Total_KW' column
df['EL_Solar_BusBarn_Total_KW'] = df['EL_Solar_BusBarn_Total_KW'].apply(lambda x: 0 if x > 60 else x)

# Save the modified DataFrame back to a CSV file
df.to_csv('modified_file.csv', index=False)

# Continue with the rest of the operations on the filtered DataFrame
filtered_df = df

# Replace repeat values with 0
repeat_mask = filtered_df.duplicated('EL_Solar_BusBarn_Total_KW', keep=False)
filtered_df.loc[repeat_mask, 'EL_Solar_BusBarn_Total_KW'] = 0

# Relabel rows with a new sequence of labels
new_labels = [f"{i + 1}" for i in range(len(filtered_df))]
filtered_df.index = new_labels

# Save the modified DataFrame to a new CSV file
filtered_df.to_csv("modified.csv")

# Filter rows based on a condition
ali_df = filtered_df[filtered_df['EL_Solar_BusBarn_Total_KW'] > 0.0001]

# Relabel rows with a new sequence of labels
new_labels = [f"{i + 1}" for i in range(len(ali_df))]
ali_df.index = new_labels

# Save the modified DataFrame to a new CSV file
ali_df.to_csv("ali_modified.csv")
ali_df


,Flow Tag Date,EL_Solar_BusBarn_Total_KW
1,2011-05-14 07:00:00,0.044322
2,2011-05-14 08:00:00,0.201312
3,2011-05-14 09:00:00,0.223929
4,2011-05-14 10:00:00,0.645141
5,2011-05-14 11:00:00,1.391401
...,...,...
64297,2023-09-21 09:00:00,5.203238
64298,2023-09-21 10:00:00,7.773371
64299,2023-09-21 11:00:00,11.066359
64300,2023-09-21 12:00:00,4.890486


In [ ]:
import pandas as pd
from datetime import datetime

# URL with the shareable link to your Google Sheets document
google_sheets_url = "https://docs.google.com/spreadsheets/d/1jdoToY-lbibVxI0mcGwPf6LkkI786JJQwDWVzQQ7Glo/edit#gid=159703025"

# Read data
df = pd.read_csv(f"{google_sheets_url.replace('/edit#gid=', '/gviz/tq?tqx=out:csv&gid=')}")

# Convert the 'Date' column to datetime format
df['Flow Tag Date'] = pd.to_datetime(df['Flow Tag Date'], format='%d-%b-%y %H:%M:%S')

# Set the 'Flow Tag Date' column as the index
df.set_index('Flow Tag Date', inplace=True)

# Replace values above 60 kW with 0 in the 'EL_Solar_BusBarn_Total_KW' column
df['EL_Solar_BusBarn_Total_KW'] = df['EL_Solar_BusBarn_Total_KW'].apply(lambda x: 0 if x > 60 else x)

# Continue with the rest of the operations on the filtered DataFrame
filtered_df = df

# Replace repeat values with 0
repeat_mask = filtered_df.duplicated('EL_Solar_BusBarn_Total_KW', keep=False)
filtered_df.loc[repeat_mask, 'EL_Solar_BusBarn_Total_KW'] = 0

# Resample the DataFrame to daily frequency and sum the values for each day
daily_df = filtered_df.resample('D').sum()

# Reset the index to have 'Flow Tag Date' as a column again
daily_df.reset_index(inplace=True)

def get_kw_for_day(dataframe, input_day):
    # Convert the input_day to datetime format
    input_day = pd.to_datetime(input_day, format='%d-%b-%y')

    # Check if the input day is in the daily DataFrame
    daily_kw = daily_df[daily_df['Flow Tag Date'].dt.date == input_day.date()]['EL_Solar_BusBarn_Total_KW'].values
    if len(daily_kw) > 0:
        print("On", input_day.date(), "the total KW for the day was:", daily_kw[0])
    else:
        print("The input day is not available in the data.")
        return 0

    # Filter the original DataFrame for the input day
    hourly_df = dataframe[dataframe.index.date == input_day.date()]

    # Calculate and print conversion efficiency for every hour of the inputted day
    hourly_df = hourly_df.resample('H').sum()
    hourly_df['Conversion Efficiency'] = hourly_df['EL_Solar_BusBarn_Total_KW'].apply(lambda x: x / 400)
    print("Hourly Conversion Efficiency on", input_day.date(), ":")
    print(hourly_df[['Conversion Efficiency']])

    return daily_kw[0] if len(daily_kw) > 0 else 0

# Take user input for the day
input_day = input("Enter the day in the format 'dd-mmm-yy': ")

# Call the function with the input_day
kwhPerDay = get_kw_for_day(filtered_df, input_day)

# Conversion efficiency final
def Conversion_Effiency(Daily_KW):
    return Daily_KW / 400

print("Assuming the solar irradiance is 4kWh/M^2 during", input_day, ", the conversion efficiency is ", Conversion_Effiency(kwhPerDay), "\n or ", (Conversion_Effiency(kwhPerDay) * 100), "%")
